$$ \LaTeX \text{ command declarations here.}
\newcommand{\R}{\mathbb{R}}
\renewcommand{\vec}[1]{\mathbf{#1}}
$$

In [1]:
# plotting
%matplotlib inline
from matplotlib import pyplot as plt;

# scientific
import numpy as np;
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.learning_curve import learning_curve
from sklearn.kernel_ridge import KernelRidge

# math
from __future__ import division


# EECS 545:  Machine Learning
## Lecture 13:  Information Theory and Exponential Families
* Instructor:  **Jacob Abernethy**
* Date:  March 7, 2016




## References

This lecture draws from following resources:
* Chapter 9, "Generalized linear models and the exponential family" of *Machine Learning, A Probabilistic- Perspective* (Kevin P. Murphy), pp 281-289.


* Recommended Additional Reading:
    * Statistical Methods for Signal Processing, by Alfred O. Hero: Sections 3.5, 4.4.2
    * MLAPP (Murphy), Section 2.5.3

## Outline
<Info Theory>

* Exponential Family
    * Introduction
    * MLE for exponential family

## Exponential Family: Introduction

* We have seen many distributions.
    * Bernoulli
    * Gaussian
    * Exponential
    * Gamma 
    
    
* It so happens that most of them belong to same family of distributions, ** exponential family **

## Introduction(2) 


* **Why is this important**?
    * only family of distributions with finite-sized sufficient statistics* (under certain regularity conditions).
    * only family of distributions for which conjugate priors exist.
    * makes the least set of assumptions subject to some user-chosen constraints.
    * core of generalized linear models and variational inference.
    

*Further reading: Section 3.5, Statistical Methods for Signal Processing, by Alfred O. Hero.

## Definition

A pdf or pmf $p(x|\theta)$, for $x = (x_1, ...., x_m)\in \mathcal{X}^m $ and $\theta \in \Theta \subseteq \R^d$, is said to be in the **exponential family**, if it can be expressed in the form of 
$$ 
\begin{align}
p (x|\theta)
&= \frac{1}{Z(\theta)} h(x) exp[\theta^T \phi(x)] \\
&= h(x) exp[\theta^T \phi(x) - A(\theta)]
\end{align}
$$

where
$$
\begin{align}
Z(\theta) &= \int_{\mathcal{X}^m} h(x)exp[\theta^T \phi(x)] dx \\
A(\theta) &= log Z(\theta)
\end{align}
$$




## Definition (2)

Explanation of Notation

* $\theta$: natural(canonical) parameters.
* $\phi(x) \in \R^d$: vector of sufficient statistics.
* $Z(\theta)$: Partition function
* $A(\theta)$: Cumulant function (log-partition function)
* $h(x)$: scaling constant


If $\phi(x) = x$, we say it is a **natural exponential family**. 


## Definition (3)

We could generalize the last equation by replacing $\theta$ with $\eta(\theta)$, where $\eta$ is a function that maps the parameters $\theta$ to the canonical paramaters $\eta = \eta(\theta)$.

$$ 
\begin{align}
p (x|\theta)
&= h(x) exp[\eta(\theta)^T \phi(x) - A(\eta(\theta))]
\end{align}
$$

* $dim(\theta) < dim(\eta(\theta))$: ** curved exponential family** (more sufficient statistics than parameters)
* $\eta(\theta) = \theta $: **canonical form** of model (default assumption in this lecture, unless stated otherwise)


## Examples

#### Bernoulli

Bernoulli $x \in {0,1}$ can be written as:
$$ Ber(x|\mu) = \mu^x(1-\mu)^{1-x} = exp(xlog(\mu) + (1-x)log(1-\mu) = exp(\phi(x)^T \theta) $$

where $\mu = P(x=1)$, $\phi(x) = [\mathcal{I}(x=0), \mathcal{I}(x=1)]$ and $\theta = [log(\mu), log(1-\mu)]$.

* This representation is over-complete
    * linear dependence between features: $1^T \phi(x) = \mathcal{I}(x=0)+ \mathcal{I}(x=1) = 1$
    * As a result, $\theta$ is not uniquely determined.

#### Bernoulli(2)

* It is common to require that the representation be **minimal**, which means there is a unique $\theta$ associated with the distribution. Hence, define

$$ Ber(x|\mu) = (1-\mu) exp[xlog(\frac{\mu}{1-\mu})] $$

* $\phi(x) = x$
* $\theta = log (\frac{\mu}{1-\mu})$: log-odds ratio
* $Z = 1/(1-\mu)$
* $ \mu = sigmoid(\theta) = \frac{1}{1+e^{-\theta}} $ 

## Examples(2)
#### Univariate Gaussian 

$$
\begin{align}
\mathcal{N}(x|\mu, \sigma^2)
&= \frac{1}{(2\pi\sigma^2)^{1/2}}exp[-\frac{1}{2\sigma^2}(x-\mu)^2] \\
&= \frac{1}{(2\pi\sigma^2)^{1/2}}exp[-\frac{x^2}{2\sigma^2}+\frac{\mu}{\sigma^2} - \frac{\mu^2}{2\sigma^2}] \\
&= \frac{1}{Z(\theta)}exp(\theta^T \phi(x))
\end{align}
$$

* $\theta = [\frac{\mu}{\sigma^2}, \frac{-1}{2\sigma^2}]^T $$\hspace{2em}$ $\phi(x) = [x, x^2]^T$
* $Z(\mu, \sigma^2) = \sqrt{2\pi}\sigma exp(\mu^2/(2\sigma^2))$
* $A(\theta) = \frac{-\theta_1^2}{4\theta_2} - \frac{1}{2}log(-2\theta_2) - \frac{1}{2}log(2\pi) $


## Non-examples

Not all distributions of interest belong to the exponential family.

* Uniform distribution $X \sim Uni(a,b)$: support depends upon parameters

* Student t-distribution*: cannot be represented in required form.



* *Further Readings: 
    1. Section 2.5.3, MLAPP (Murphy) 
    2. Wiki article (https://en.wikipedia.org/wiki/Student%27s_t-distribution)

## Log-Partition Function

Derivatives of the log partition function can be used to generate **cumulants** of the sufficient statistics ((Hence, the name **cumulant function**)

* Moments: $\mathbf{E}(X)$, $\mathbf{E}(X^2)$
* Cumulants: $\mathbf{E}(X)$, $Var(X)$


$A(\theta)$ is **convex** function. Proof for 1-parameter distribution follows. 
    *Try it yourself*: generalize the proof to a K-parameter distribution.

#### Proof of Convexity: First derivative

$$
\begin{align}
\frac{dA}{d\theta}
&= \frac{d}{d\theta}(log \int exp(\theta\phi(x))h(x)dx) \\
&= \frac{\frac{d}{d\theta} \int exp(\theta\phi(x))h(x)dx)}{\int exp(\theta\phi(x))h(x)dx)} \\
&= \frac{\int \phi(x)exp(\theta\phi(x))h(x)dx}{exp(A(\theta)} \\
&= \int \phi(x) exp[\theta\phi(x)-A(\theta)] h(x) dx \\
&= \int \phi(x) p(x) dx \\
&= \mathbf{E}[\phi(x)]
\end{align}
$$

#### Proof of Convexity: Second derivative

$$
\begin{align}
\frac{d^2A}{d\theta^2}
& = \int \phi(x)exp[\theta \phi(x) - A(\theta)] h(x) (\phi(x) - A'(\theta)) dx \\
& = \int \phi(x) p(x) (\phi(x) - A'(\theta))dx \\
& = \int \phi^2(x) p(X) dx - A'(\theta) \int \phi(x)p(x)dx \\
& = \mathbf{E}[\phi^2(x)] - \mathbf{E}[\phi(x)]^2  \hspace{2em}   (\because A'(\theta) = \mathbf{E}[\phi(x)])  \\ 
& = var(\phi(x))
\end{align}
$$

#### Proof of Convexity: Second derivative(2)

For multi-variate case, we have 

$$ \frac{\partial^2A}{\partial\theta_i \partial\theta_j} = \mathbf{E}[\phi_i(x)\phi_j(x)] - \mathbf{E}[\phi_i(x)]\mathbf{E}[\phi_j(x)]$$

and hence,
$$ \nabla^2A(\theta) = cov[\phi(x)] $$

Since covariance is positive definite, we have $A(\theta)$ convex as required.

## Log Partition Example
#### Bernoulli

* $ A(\theta) = log(1+e^\theta)$ 
* $$ \mathbf{E}[\phi(x)] = \frac{dA}{d\theta} = \frac{e^\theta}{1+ e^\theta} = sigmoid(\theta) = \mu $$

* $$
\begin{align}
var[\phi(x)] \
&= \frac{d}{d\theta}(1+e^{-\theta})^{-1} = (1+e^{-\theta})^{-2}\cdot e^{-\theta} \\
&= \frac{e^{-\theta}}{1+e^{-\theta}}\cdot \frac{1}{1+e^{-\theta}} = \frac{1}{1+e^\theta}\cdot\frac{1}{1+e^{-\theta}} \\
&= (1-\mu)\mu
\end{align}
$$

## MLE for exponential family

#### General form of Likelihood function for Exponential Family

$$ p(\mathcal{D}|\theta) = [\prod_{i=1}^{N}h(x_i)]g(\theta)^N exp(\eta(\theta)^T[\sum_{i=1}^{N}\phi(x_i)]) $$

* Sufficient statistics:
    * N
    * $\phi(\mathcal{D}) = [\sum_{i=1}^{N}\phi_1(x_i), \dots , \sum_{i=1}^{N}\phi_k(x_i)]$


* Example:
    * Bernoulli: N, $\phi = [\sum_i \mathcal{I}(x_i=1)]$
    * Univariate Guassian: N, $\phi = [\sum_i x_i, \sum_i x_i^2]$


## Pitman-Koopman-Darmois Theorem 

Among families of probability distributions whose domain does not vary with the parameter being estimated, only in exponential families is there a sufficient statistic whose dimension remains bounded as sample size increases.

Suppose $X_n, n = 1, 2, 3, \dots$ are independent identically distributed random variables whose distribution is known to be in some family of probability distributions. Only if that family is an exponential family is there a (possibly vector-valued) sufficient statistic $T(X_1, \dots, X_n)$ whose number of scalar components does not increase as the sample size $n$ increases.

* **Sufficiency of statistic restricts the type of distribution.**

### PKD Theorem (2)

* One of the conditions required in this theorem is that the support of the distribution not be dependent on the parameter.
    * Example: 
        * $p(x|\theta) = U(x|\theta) = \frac{1}{\theta}\mathcal{I}(0 \leq x \leq \theta)$
        * Likelihood: $ p(\mathcal{D}|\theta) = \theta^{-N}\mathcal{I}(0 \leq max_i\{x_i\} \leq \theta) $
        * sufficient statistics: $N$ and $s(\mathcal{D}) = max_i \{x_i\}$
        * Finite in size, but support depends on $\theta$ and hence, not in exponential family.

## Computing MLE for Canonical model

* Given $N$ iid data points $\mathcal{D} = (x_1, \dots , x_N))$


* log-likelihood: $ log p(\mathcal{D}|\theta) = \theta^T \phi(\mathcal{D}) - N A(\theta) $

    * $- A(\theta)$ is concave in $\theta$.
    * $\theta^T\phi(\mathcal{D})$ is linear in $\theta$.
    * Consequently, log-likelihood is **concave** and achieves global maximum.


## Method of Moments (MoM)

* To derive MLE, use the fact that derivative of log-partition function yields $\mathbf{E}[\phi(x)]$.

$$
\begin{align}
\nabla_\theta log p(\mathcal{D}|\theta) = 0\\
\implies \phi(\mathcal{D}) - N \mathbf{E}[\phi(X)] = 0\\
\implies \mathbf{E}[\phi(X)] = \frac{\phi (\mathcal{D})}{N}\\
\implies \mathbf{E}[\phi(X)] = \frac{1}{N}\sum_{i=1}^{N}\phi(x_i)
\end{align}
$$

* *The sample of the sufficient statistics must equal the model’s theoretical expected sufficient statistics*.

## Method of Moments (2)

* This is called **moment matching** or method of moments*.

* Example: Bernoulli Distribution
    * $\phi(x) = \mathcal{I}(X=1)$
    * $\mathbf{E}[\phi(X)] = p(X=1) = \hat \mu = \frac{1}{N} \sum_{i=1}^{N} \mathcal{I}(x_i=1)$





*Further Reading: Section 4.4.2, Statistical Methods for Signal Processing, by Alfred O. Hero

## Bayes for Exponential Family

* Fact: Exact Bayesian analysis is considerably simplified if the prior is conjugate to the likelihood.

* Simply, this means that prior $p(\mathcal{D}|\tau)$ has the same form as likelihood $p(\mathcal{D}|\theta)$.

* This requires likelihood to have finite sufficient statistics ,such that $p(\mathcal{D}|\theta) = p(s(\mathcal{D})|\theta)$

* Hence, Exponential family to the rescue!


**Running Example**: Bernoulli distribution, $Ber(x|\theta) = \theta^x (1-\theta)^{1-x}$

### Likelihood

Likelihood: 
$$ p(\mathcal{D}|\theta) \propto g(\theta)^N exp[\eta(\theta)^T s_N]\\
s_N = \sum_{i=1}^{N}s(x_i)$$

In terms of canonical parameters:
$$ p(\mathcal{D}|\eta) \propto exp[N\eta^T \bar{s} -N A(\eta)] \\
\bar s = \frac{1}{N}s_N $$


**Example**:
$$p(\mathcal{D}|\theta) = (1-\theta)^N exp[log(\frac{\theta}{1-\theta})\sum_i x_i]$$

## Prior

Prior: 
$$ p(\theta| \nu_0, \tau_0) \propto g(\theta)^{\nu_0} exp[\eta(\theta)^T \tau_0] $$

* Denote $ \tau_0 = \nu_0 \bar{\tau}_0$ to separate out the size of the prior pseudo-data, $\nu_0$ , from the mean of the sufficient statistics on this pseudo-data, $\tau_0$ . Hence,

$$ p(\theta| \nu_0, \bar \tau_0) \propto exp[\nu_0\eta^T \bar \tau_0  - \nu_0 A(\eta)] $$

## Prior(2)

**Example**: 

$$
\begin{align}
p(\theta| \nu_0, \tau_0) 
&\propto (1-\theta)^{\nu_0} exp[log(\frac{\theta}{1-\theta}\tau_0)] \\
&= \theta^{\nu_0}(1-\theta)^{\nu_0 - \tau_0}
\end{align}
$$

Define $\alpha = \tau_0 +1 $ and $\beta = \nu_0 - \tau_0 +1$ to see that this is a *beta distribution*.

## Posterior

* Posterior: 
$$ p(\theta|\mathcal{D}) = p(\theta|\nu_N, \tau_N) = p(\theta| \nu_0 +N, \tau_0 +s_N) $$

* Note that we obtain hyper-parameters by adding. Hence,

$$ \begin{align}
p(\eta|\mathcal{D})
&\propto exp[\eta^T (\nu_0 \bar\tau_0 + N \bar s) - (\nu_0 + N) A(\eta) ] \\
&= p(\eta|\nu_0 + N, \frac{\nu_0 \bar\tau_0 + N \bar s}{\nu_0 + N}
\end{align}$$

* *posterior hyper-parameters are a convex combination of the prior mean hyper-parameters and the average of the sufficient statistics.*

## Posterior(2)

**Example**:

* Define sufficient statistic $s = \sum_i\mathcal{I}(x_i=1)$:

$$
\begin{align}
p(\theta| \mathcal{D}) 
&\propto \theta^{\tau_0 +s} (1-\theta)^{\nu_0- \tau_0 + n - s} \\
&= \theta^{\tau_n}(1-\theta)^{\nu_n - \tau_n}
\end{align}
$$